Podemos simular varias moléculas o granos, como en un gas, y hacer que estos interactúen entre sí. Para ello, modelaremos la fuerza de contacto entre las moléculas utilizando el modelo de fuerzas de Hertz. A continuación, haremos una simulación de un gas en una caja, utilizando como algoritmo de integración del método de PEFRL que hemos desarrollado previamente:

In [5]:
%%writefile Gas2D.cpp

#include <iostream>
#include <cmath>
#include "vector.h"
#include "Random64.h"
using namespace std;


// Constantes del problema físico:
const double G=1;
const int Nx=5, Ny=5;  // Quiero crear un gas, con un total de 25 moléculas distribuidas uniformemente dentro de una caja
const int N=Nx*Ny;  // Número total de moléculas
const double KHertz=1.0e4;
double Lx=60, Ly=60;  // Tamaño de la caja

// Constantes del algoritmo de integración
const double Xi = 0.1786178958448091;
const double Lambda = -0.2123418310626054;
const double Chi = -0.06626458266981849;
const double Um2LambdaU2 = (1-2*Lambda)/2;
const double Um2ChipXi = 1-2*(Chi+Xi);

// Declaración de las clases:
class Cuerpo;
class Colisionador;

// Declaración de las interfases de las clases:
class Cuerpo{
  private:
    vector3D r,V,F; double m,R;
  public:
    void Inicie(double x0,double y0,double z0,double Vx0,double Vy0,double Vz0,
	        double m0,double R0);
    void BorreFuerza(void){F.load(0,0,0);}; // Inline
    void SumeFuerza(vector3D dF){F+=dF;}; // Inline
    void Mueva_r(double dt, double coef);
    void Mueva_v(double dt, double coef);
    void Dibujese(void);
    double Getx(void){return r.x();}; // Inline
    double Gety(void){return r.y();}; // Inline
    friend class Colisionador;    // De esta manera, hacemos que la clase Colisionador pueda tener acceso a los datos privados de la clase Cuerpo
};
class Colisionador{
  private:
  public:
    void CalculeTodasLasFuerzas(Cuerpo * Moleculas);
    void CalculeFuerzaEntre(Cuerpo & Molecula1,Cuerpo & Molecula2);
};

// Implementación de las funciones de las clases:
// Funciones de la clase Cuerpo:
void Cuerpo::Inicie(double x0,double y0,double z0,double Vx0,double Vy0,double Vz0,
	      double m0,double R0){
  r.load(x0,y0,z0); V.load(Vx0,Vy0,Vz0); m=m0; R=R0;
}
void Cuerpo::Mueva_r(double dt, double coef){
  // Algoritmo de Forest-Ruth
  r+=(coef*dt)*V;
}
void Cuerpo::Mueva_v(double dt, double coef){
  // Algoritmo de Forest-Ruth
  V+=(coef*dt/m)*F;
}
void Cuerpo::Dibujese(void){
  cout<<" , "<<r.x()<<"+"<<R<<"*cos(t),"<<r.y()<<"+"<<R<<"*sin(t)";
}

// Funciones de la clase Colisionador:
void Colisionador::CalculeTodasLasFuerzas(Cuerpo * Molecula){
  int i,j;
  // 1) Borramos las fuerzas de todos las moléculas:
  for(i=0;i<N+4;i++){
    Molecula[i].BorreFuerza();
  }
  // 2) Se recorre por parejas la clase Cuerpo creada, se calcula la fuerza de cada pareja y se suma a cada molécula:
  for(i=0;i<N;i++){
    for(j=i+1;j<N+4;j++){
      CalculeFuerzaEntre(Molecula[i],Molecula[j]);
    }
  }

}
void Colisionador::CalculeFuerzaEntre(Cuerpo & Molecula1,Cuerpo & Molecula2){
  // Determinamos si hay colisión:
  vector3D r21 = Molecula2.r-Molecula1.r; double r = r21.norm();
  double R1 = Molecula1.R, R2 = Molecula2.R;
  double s=(R1+R2)-r;
  // Condición de colisión:
  if(s>0){
    // Calculamos el vector normal:
    vector3D n = (1.0/r)*r21;
    // Calculamos la fuerza:
    vector3D F2 = n*(KHertz*pow(s,1.5));
    // Sumo las fuerzas a las moléculas:
    Molecula2.SumeFuerza(F2);  Molecula1.SumeFuerza(F2*(-1));
  }


}

//----------- Funciones Globales -----------

//Funciones de animación:
void InicieAnimacion(void){
  //cout<<"set terminal gif animate"<<end;
  //cout<<"set output 'Gas2D.gif'"<<end;
  cout<<"unset key"<<endl;
  cout<<"set xrange[-10:"<<Lx+10<<"]"<<endl;
  cout<<"set yrange[-10:"<<Ly+10<<"]"<<endl;
  cout<<"set size ratio -1"<<endl;
  cout<<"set parametric"<<endl;
  cout<<"set trange[0:7]"<<endl;
  cout<<"set isosamples 12"<<endl;
}

void InicieCuadro(void){
  cout<<"plot 0,0 ";
  cout<<" , "<<Lx/7<<"*t,0";  // Pared de abajo
  cout<<" , "<<Lx/7<<"*t,"<<Ly;  // Pared de arriba
  cout<<" , 0,"<<Ly/7<<"*t";  // Pared de la izquierda
  cout<<" , "<<Lx<<","<<Ly/7<<"*t";  // Pared de la derecha
}

void TermineCuadro(void){
  cout<<endl;
}

int main(){

  Cuerpo Molecula[N+4];  // Tendremos un gas de N moléculas, pero las 4 que añadimos de manera extra van a sernos útiles para definir las paredes de la caja, y por lo tanto, habrá interacción entre la caja y el gas
  Colisionador Newton;
  Crandom ran64(12);
  int i,j;
  // Parámetros de la simulación:
  double m0=1.0, R0=2.0;
  // Variables auxiliares para la condición inicial y correr la simulación:
  double k_BT=10, V0=sqrt(k_BT/m0); // Definimos una temperatura y, mediante el teorema del virial, definimos una velocidad inicial
  double t, dt=1e-3, ttotal=10*(Lx/V0);
  int Ncuadros=1000; double tdibujo,tcuadro=ttotal/Ncuadros;
  double dx=Lx/(Nx+1), dy=Ly/(Ny+1);
  double theta;
  double x0,y0,Vx0,Vy0;
  // Variables auxiliares para las paredes:
  double Rpared=100*Lx, Mpared=100*m0;

  InicieAnimacion();
  // INICIO:
  // Inicializamos las paredes:
  //----------------(x0,y0,z0,Vx0,Vy0,Vz0,m0,R0)
  Molecula[N].Inicie(Lx/2, Ly+Rpared, 0, 0, 0, 0, Mpared, Rpared); // Pared de arriba
  Molecula[N+1].Inicie(Lx/2, -Rpared, 0, 0, 0, 0, Mpared, Rpared); // Pared de abajo
  Molecula[N+2].Inicie(Lx+Rpared, Ly/2, 0, 0, 0, 0, Mpared, Rpared); // Pared de la derecha
  Molecula[N+3].Inicie(-Rpared, Ly/2, 0, 0, 0, 0, Mpared, Rpared); // Pared de la izquierda
  // Inicializamos las moléculas:
  for(i=0;i<Nx;i++)
    for(j=0;j<Ny;j++){
      theta=2*M_PI*ran64.r();
      x0=(i+1)*dx; y0=(j+1)*dy; Vx0=V0*cos(theta); Vy0=V0*sin(theta);
      //----------------(x0,y0,z0,Vx0,Vy0,Vz0,m0,R0)
      Molecula[j*Nx+i].Inicie(x0, y0, 0, Vx0, Vy0, 0, m0, R0);
  }
  // CORRE PROGRAMA:
  for(t=tdibujo=0;t<ttotal;t+=dt,tdibujo+=dt){
    if(tdibujo>tcuadro){
      //cout<<Molecula[0].Getx()<<" "<<Molecula[0].Gety()<<endl;
      //cout<<Molecula[1].Getx()<<" "<<Molecula[1].Gety()<<endl;
      InicieCuadro();
      for(i=0;i<N;i++) Molecula[i].Dibujese();
      TermineCuadro();
      tdibujo=0;
    }
    /* Ahora el cálculo de las fuerzas depende de la clase colisionador y no de la clase cuerpo. Además, recordemos que estamos
       aplicando el método de integración PEFRL de manera simultánea para los N cuerpos que tengamos.*/

    for(i=0;i<N;i++) Molecula[i].Mueva_r(dt,Xi);
    Newton.CalculeTodasLasFuerzas(Molecula);  for(i=0;i<N;i++) Molecula[i].Mueva_v(dt,Um2LambdaU2);
    for(i=0;i<N;i++) Molecula[i].Mueva_r(dt,Chi);
    Newton.CalculeTodasLasFuerzas(Molecula);  for(i=0;i<N;i++) Molecula[i].Mueva_v(dt,Lambda);
    for(i=0;i<N;i++) Molecula[i].Mueva_r(dt,Um2ChipXi);
    Newton.CalculeTodasLasFuerzas(Molecula);  for(i=0;i<N;i++) Molecula[i].Mueva_v(dt,Lambda);
    for(i=0;i<N;i++) Molecula[i].Mueva_r(dt,Chi);
    Newton.CalculeTodasLasFuerzas(Molecula);  for(i=0;i<N;i++) Molecula[i].Mueva_v(dt,Um2LambdaU2);
    for(i=0;i<N;i++) Molecula[i].Mueva_r(dt,Xi);

  }
  return 0;
}

Overwriting Gas2D.cpp


In [6]:
%%shell
g++ Gas2D.cpp
./a.out > "Instrucciones - Gas2D.txt"